In [1]:
# Third test for git aliases (e.g., git-update-plus, git-next-checkin, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [ ]:
## TODO2: put common setup in helper script
## ex: source git-aliases-tests-common.sh

In [ ]:
# Global Setup
# Creating temp local repos to test on
## TODO2: Use something like TEMP_TEST_DIR becuase you are not changing system temp directory.
TMP=/tmp/temp-git-test
git clone https://github.com/tomasohara/git-bash-test.git "$TMP" > /dev/null 2>&1

In [ ]:
## BAD
## git clone https://github.com/LorenzoFerraces/bash-test.git "$TMP"
##
## # Move to second local to push some changes
## command cd "$TMP" 
## git pull origin main 
## echo "hello second local" > hello.txt
## git add hello.txt
## git commit -m "first commit on second local"
## git push



In [ ]:
## TODO1: fix changes so that all temp files use "temp-" prefix
## NOTE: Please write these test scripts with an eye towards Murphy's Law!
## ...   That is, what can go wrong will go wrong!
## NOTE: bye.txt changed to temp-bye.txt, etc.

In [ ]:
## TODO1 Fix this following git-aliases-tests-1 as requested
## Also follow git-aliases-tests-2.
##
## Use following Bash snippet to output .sh versions to facilitate comparisons
## (e.g., git-aliases-tests-1 vs git-aliases-tests-2 vs git-aliases-tests-3).
##
## note:
## - [batch] converting all Jupyter bash tests to .sh format (i.e., without expected output)
##  
## $ {
##   # pre-init: CONV_OPTIONS=""; CONV_EXTENSION="batspp";
##   # pre-init: CONV_GLOB="*tips*"
##   conv_options="${CONV_OPTIONS:-"--just-code"}";
##   conv_extension="${CONV_EXTENSION:-"sh"}";
##   conv_glob="${CONV_GLOB:-"*"}";
##   #
##   mkdir -p tests/output-files;
##   for f in tests/${conv_glob}.ipynb; do
##       b=$(basename $f .ipynb);
##       TRACE_INVOCATION=0 jupyter_to_batspp.py $conv_options --stdout $f >| tests/output-files/"$b.$conv_extension";
##   done;
## }
## 
## $ pwd
## /home/tomohara/programs/bash/tom-shell-scripts

In [ ]:
# git-diff-plus tests
cat temp-hello.txt > temp-bye.txt
echo "differ" >> temp-hello.txt
diffs=$(git-diff-plus temp-hello.txt temp-bye.txt | grep "^\+\w" | wc -l)

In [4]:
# Assert 
# there should only be one line of difference
[ $diffs -eq 1 ]; echo $?;

0


In [5]:
# git-diff-list tests
changed=$(git-diff-list | wc -l)

In [6]:
# Assert 
# there should be one file changed
[ $changed -eq 1 ]; echo $?;

0


In [7]:
# git-vdiff-alias tests
# TODO2: what is nvim???
killall nvim > /dev/null
git-vdiff-alias 

instances=$(ps -e | grep "nvim" | wc -l)

nvim: no process found
[1] 15207

[1]+  Stopped                 git-difftool-plus "$@"


In [8]:
# Assert there is an instance of nvim 
[ $instances -eq 1 ]; echo $?;

0


In [9]:
# git-next-checkin tests
rm -f temp-bye.txt
previous_changes=$( git status -s | wc -l )

removed 'bye.txt'


In [ ]:
# set env variables and run git-next-checkin
export GIT_MESSAGE="checkin"
previous=$GIT_NO_CONFIRM
export GIT_NO_CONFIRM="1"
git-next-checkin temp-hello.txt

fatal: Unable to read current working directory: No such file or directory
[2]-  Exit 128                ( sleep $delay; git-difftool-plus "$mod_file" )  (wd: /tmp/temp-git-test)
(wd now: /tmp)


In [ ]:
## TODO1: fixme!
# delete new files created by git-next-cheking and count changes
## BAD: rm -rf $(ls | grep '[.][0-9]*[.]')
## TODO: changes=$( git status -s | wc -l )
## NOTE: *** forces test to fail, so  dangerous 'rm -rf' above fixed ***
changes=-1

In [11]:
# Assert there were changes before, and no changes now
[ $previous_changes -gt 1 ] && [ $changes -eq 0 ]; echo $?;

0
0


In [12]:
# undo changes and remove newly created files

# TODO2:
## also, converting unique ID's to codes like temp-git-test/.git/objects/pack/pack-HEX.idx
## EX: cat $log | perl -pe 's/\b[0-9]+\b/NUM/g; s/\b[a-f0-9]+\b/HEX/g;'

## TODO1: be more diligent about timely changes (FIXME asap!)
## BAD: rm -f *.txt > /dev/null
## BAD: rm -rf $(ls | grep '[.][0-9]*[.]')

# Remove temporary git-log files
# TODO1: use .gitingore!
rm -f _*[0-9]*.log


[main dca425a] cleanup test files
 1 file changed, 2 deletions(-)
 delete mode 100644 hello.txt
Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 4 threads
Compressing objects: 100% (1/1), done.
Writing objects: 100% (2/2), 245 bytes | 245.00 KiB/s, done.
Total 2 (delta 0), reused 1 (delta 0), pack-reused 0
To https://github.com/LorenzoFerraces/bash-test.git
   fa905c1..dca425a  main -> main


In [ ]:
git rm -f temp-*.txt > /dev/null
git add . > /dev/null
git commit -m "cleanup test files"
git push

In [2]:
# delete test local repos

command cd "/tmp"


In [3]:
## TODO1: rename using timestamp (e.g., to allow for posthoc analysis)!
## BAD: rm -rf temp-git-test > /dev/null

In [6]:
## TODO: drop (no need to cleanup up environment in notebook)
export GIT_NO_CONFIRM=$previous